In [ ]:
import zea
import matplotlib.pyplot as plt
import jax
import numpy as np
import ulsa.ops
from tqdm import tqdm
from ulsa.pipeline import beamforming, Pipeline

zea.init_device()

In [ ]:
acq = "20251222_s3_a4ch_line_dw_0000"

In [ ]:
path = f"/mnt/z/usbmd/Wessel/Verasonics/2026_USBMD_A4CH_S51/{acq}.hdf5"

file = zea.File(path)
frame_nr = -1
scan = file.scan()
from ulsa.utils import update_scan_for_polar_grid

scan.set_transmits("focused")
update_scan_for_polar_grid(scan)

# Avoid the first pixel being at r = 0
# dr = scan.zlims[1] - scan.zlims[0] / scan.grid_size_z
# scan.zlims = (scan.zlims[0] + dr, scan.zlims[1] + dr)

width = 2e6  # Hz
f1 = scan.demodulation_frequency - width / 2
f2 = scan.demodulation_frequency + width / 2
bandpass_rf = zea.func.get_band_pass_filter(128, scan.sampling_frequency, f1, f2)

raw_data = file.load_data("raw_data", (frame_nr, scan.selected_transmits))

In [ ]:
rx_apo = ulsa.ops.lines_rx_apo(
    n_tx=scan.n_tx, grid_size_z=scan.grid_size_z, grid_size_x=scan.grid_size_x
)

pipeline = Pipeline(beamforming(rx_apo=True), with_batch_dim=False)
# pipeline.append(zea.ops.ScanConvert(order=1))

params = pipeline.prepare_parameters(
    scan=scan, bandpass_rf=bandpass_rf, bandwidth=2e6, rx_apo=rx_apo, minval=0
)
output = pipeline(data=raw_data, **params)
data = output["data"]

plt.figure(figsize=(10, 5))
plt.imshow(
    data,
    cmap="gray",
    vmin=-60,
    vmax=0,
    interpolation="nearest",
    # extent=scan.extent * 1e3,
)
plt.savefig("processed_image.png", dpi=300, bbox_inches="tight")

In [ ]:
def get_first_frame(file, scan):
    top_cropping = 50
    pipeline = Pipeline(beamforming(rx_apo=True), with_batch_dim=False)
    pipeline.append(zea.ops.keras_ops.ExpandDims(axis=-1))
    pipeline.append(zea.ops.LeeFilter(sigma=5))
    pipeline.append(zea.ops.keras_ops.Squeeze(axis=-1))

    data_array, _ = pipeline.run(
        file,
        scan,
        rx_apo=rx_apo,
        bandpass_rf=bandpass_rf,
        bandwidth=2e6,
        minval=0,
    )

    data_array = zea.func.translate(data_array, (-60, 0), (0, 1))
    data_array = np.clip(data_array, 0, 1)

    diff = np.abs(np.diff(data_array, axis=0))[:, top_cropping:].mean(axis=(1, 2))
    return diff.argmax() + 1, diff


first_frame_idx, _ = get_first_frame(file, scan)
first_frame_idx

In [ ]:
n_frames = file.n_frames
indices = np.arange(first_frame_idx, first_frame_idx + n_frames) % n_frames

raw_data_sequence = []
for idx in tqdm(indices):
    _data = file.load_data("raw_data", (idx, scan.selected_transmits))
    raw_data_sequence.append(_data)

In [ ]:
name = file.stem + "_zea.mp4"
pipeline = Pipeline(beamforming(rx_apo=False), with_batch_dim=False)
pipeline.append(zea.ops.ScanConvert(order=1))
pipeline.to_video_file(
    file.path.parent / name,
    data=raw_data_sequence,
    scan=scan,
    minval=0,
    bandwidth=2e6,
    bandpass_rf=bandpass_rf,
)

In [ ]:
from in_house_cardiac.cardiac_scan import cardiac_scan

images, _ = cardiac_scan(path)
plt.imshow(images[0], cmap="gray", vmin=-60, vmax=0)

In [ ]:
plt.figure(figsize=(10, 5))

if "non_standard_elements/verasonics_image_buffer" in file:
    vera_image = file["non_standard_elements/verasonics_image_buffer"][frame_nr][0].T
else:
    vera_image = file.load_data("image", frame_nr)
plt.imshow(vera_image, cmap="gray", vmin=-60, vmax=0)
# plt.savefig("vera_image.png", dpi=300)

In [ ]:
vera_images = file.load_data("image")
vera_images = np.clip(vera_images, a_min=-60, a_max=0)
vera_images = zea.display.to_8bit(vera_images, (-60, 0), pillow=False)
name = file.stem + "_vera.mp4"
zea.io_lib.save_video(vera_images, file.path.parent / name)